# Concurrent Workflow with Visualization Demo

This notebook demonstrates a **fan-out/fan-in workflow** pattern using multiple AI agents:

## Workflow Structure
1. **Dispatcher** - Receives a prompt and fans out to three domain experts
2. **Three Expert Agents** (parallel execution):
   - 🔬 **Researcher** - Market and product research
   - 📢 **Marketer** - Marketing strategy and messaging
   - ⚖️ **Legal** - Compliance and legal review
3. **Aggregator** - Collects and consolidates all expert responses

## Visualization
We'll generate both Mermaid and GraphViz diagrams to visualize the workflow structure.

## Prerequisites
- Azure AI / Azure OpenAI configured
- `az login` completed for authentication
- `.env` file with Azure credentials

## Step 1: Setup and Imports
Load required libraries and environment variables.

In [1]:
import asyncio
import os
import sys
from dataclasses import dataclass
from pathlib import Path
from typing_extensions import Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    AgentRunEvent,
    ChatMessage,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    WorkflowViz,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv

# Detect if running in Jupyter and adjust path accordingly
try:
    import ipykernel
    # Running in notebook: go up 3 parents (visualization -> workflow -> getting_started -> samples -> python)
    parent_levels = 3
except ImportError:
    # Running as script: go up 4 parents
    parent_levels = 4

# Load environment variables from .env file
if '__file__' in globals():
    env_path = Path(__file__).resolve().parents[parent_levels] / ".env"
else:
    # In notebook, use current working directory as reference
    env_path = Path.cwd().parents[parent_levels] / ".env"

load_dotenv(dotenv_path=env_path, override=True)

# Verify environment variables are loaded
required_vars = ['AZURE_AI_PROJECT_ENDPOINT', 'AZURE_AI_MODEL_DEPLOYMENT_NAME']
for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✓ {var} is set")
    else:
        print(f"✗ {var} is NOT set")

print(f"\n✓ Environment loaded from: {env_path}")

✓ AZURE_AI_PROJECT_ENDPOINT is set
✓ AZURE_AI_MODEL_DEPLOYMENT_NAME is set

✓ Environment loaded from: /Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/.env


## Step 2: Define Custom Executors

### Dispatcher Executor
Fans out the prompt to all expert agents.

In [2]:
class DispatchToExperts(Executor):
    """Dispatches the incoming prompt to all expert agent executors (fan-out)."""

    def __init__(self, expert_ids: list[str], id: str | None = None):
        super().__init__(id=id or "dispatch_to_experts")
        self._expert_ids = expert_ids

    @handler
    async def dispatch(self, prompt: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        # Wrap the incoming prompt as a user message for each expert and request a response.
        initial_message = ChatMessage(Role.USER, text=prompt)
        print(f"📤 Dispatching to {len(self._expert_ids)} experts...")
        for expert_id in self._expert_ids:
            print(f"  → Sending to: {expert_id}")
            await ctx.send_message(
                AgentExecutorRequest(messages=[initial_message], should_respond=True),
                target_id=expert_id,
            )

print("✓ DispatchToExperts class defined")

✓ DispatchToExperts class defined


### Aggregator Executor
Collects responses from all experts and consolidates them.

In [3]:
@dataclass
class AggregatedInsights:
    """Structured output from the aggregator."""
    research: str
    marketing: str
    legal: str


class AggregateInsights(Executor):
    """Aggregates expert agent responses into a single consolidated result (fan-in)."""

    def __init__(self, expert_ids: list[str], id: str | None = None):
        super().__init__(id=id or "aggregate_insights")
        self._expert_ids = expert_ids

    @handler
    async def aggregate(self, results: list[AgentExecutorResponse], ctx: WorkflowContext[Never, str]) -> None:
        print(f"\n📥 Aggregating {len(results)} expert responses...")
        
        # Map responses to text by executor id for a simple, predictable demo.
        by_id: dict[str, str] = {}
        for r in results:
            # AgentExecutorResponse.agent_run_response.text contains concatenated assistant text
            by_id[r.executor_id] = r.agent_run_response.text
            print(f"  ← Received from: {r.executor_id}")

        research_text = by_id.get("researcher", "")
        marketing_text = by_id.get("marketer", "")
        legal_text = by_id.get("legal", "")

        aggregated = AggregatedInsights(
            research=research_text,
            marketing=marketing_text,
            legal=legal_text,
        )

        # Provide a readable, consolidated string as the final workflow result.
        consolidated = (
            "Consolidated Insights\n"
            "====================\n\n"
            f"Research Findings:\n{aggregated.research}\n\n"
            f"Marketing Angle:\n{aggregated.marketing}\n\n"
            f"Legal/Compliance Notes:\n{aggregated.legal}\n"
        )

        await ctx.yield_output(consolidated)

print("✓ AggregateInsights class defined")

✓ AggregateInsights class defined


## Step 3: Create Expert Agents
Initialize the three domain expert agents with specialized instructions.

In [4]:
# Create agent executors for domain experts
print("Creating Azure OpenAI chat client...")
chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

print("\nCreating expert agents...")

researcher = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
            " opportunities, and risks."
        ),
    ),
    id="researcher",
)
print("  🔬 Researcher agent created")

marketer = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
            " aligned to the prompt."
        ),
    ),
    id="marketer",
)
print("  📢 Marketer agent created")

legal = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
            " based on the prompt."
        ),
    ),
    id="legal",
)
print("  ⚖️  Legal agent created")

expert_ids = [researcher.id, marketer.id, legal.id]
print(f"\n✓ All {len(expert_ids)} expert agents ready!")

Creating Azure OpenAI chat client...

Creating expert agents...
  🔬 Researcher agent created
  📢 Marketer agent created
  ⚖️  Legal agent created

✓ All 3 expert agents ready!


## Step 4: Build the Workflow
Connect all executors in a fan-out/fan-in pattern.

In [5]:
# Create dispatcher and aggregator
dispatcher = DispatchToExperts(expert_ids=expert_ids, id="dispatcher")
aggregator = AggregateInsights(expert_ids=expert_ids, id="aggregator")

# Build the workflow
print("Building workflow...")
workflow = (
    WorkflowBuilder()
    .set_start_executor(dispatcher)
    .add_fan_out_edges(dispatcher, [researcher, marketer, legal])
    .add_fan_in_edges([researcher, marketer, legal], aggregator)
    .build()
)

print("✓ Workflow built successfully!")
print("\nWorkflow structure:")
print("  1. dispatcher (start)")
print("  2. Fan-out to: researcher, marketer, legal (parallel)")
print("  3. Fan-in to: aggregator (consolidate)")

Building workflow...
✓ Workflow built successfully!

Workflow structure:
  1. dispatcher (start)
  2. Fan-out to: researcher, marketer, legal (parallel)
  3. Fan-in to: aggregator (consolidate)


## Step 5: Visualize the Workflow

### Mermaid Diagram
Generate a Mermaid diagram showing the workflow structure.

In [6]:
# Create visualization
viz = WorkflowViz(workflow)

# Display Mermaid diagram
print("Mermaid Diagram:")
print("=" * 60)
mermaid_str = viz.to_mermaid()
print(mermaid_str)
print("=" * 60)
print("\n💡 Copy the above diagram to https://mermaid.live to visualize!")

Mermaid Diagram:
flowchart TD
  dispatcher["dispatcher (Start)"];
  researcher["researcher"];
  marketer["marketer"];
  legal["legal"];
  aggregator["aggregator"];
  fan_in__aggregator__e3a4ff58((fan-in))
  legal --> fan_in__aggregator__e3a4ff58;
  marketer --> fan_in__aggregator__e3a4ff58;
  researcher --> fan_in__aggregator__e3a4ff58;
  fan_in__aggregator__e3a4ff58 --> aggregator;
  dispatcher --> researcher;
  dispatcher --> marketer;
  dispatcher --> legal;

💡 Copy the above diagram to https://mermaid.live to visualize!


###  Render via Mermaid.ink API

Use the Mermaid.ink service to render the diagram as an image (works in all environments).

In [12]:
from IPython.display import Image, display
import base64
import json

# Method 2: Using Mermaid.ink API (more reliable)
def display_mermaid_via_api(mermaid_code):
    """Display Mermaid diagram using Mermaid.ink API."""
    # Encode the Mermaid code
    graphbytes = mermaid_code.encode("utf-8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    
    # Create URL for Mermaid.ink
    image_url = f"https://mermaid.ink/img/{base64_string}"
    
    print(f"📊 Mermaid Diagram (via mermaid.ink):\n")
    print(f"🔗 Direct link: {image_url}\n")
    
    # Display the image
    display(Image(url=image_url, width=800))

# Display using the API method
display_mermaid_via_api(mermaid_str)

📊 Mermaid Diagram (via mermaid.ink):

🔗 Direct link: https://mermaid.ink/img/Zmxvd2NoYXJ0IFRECiAgZGlzcGF0Y2hlclsiZGlzcGF0Y2hlciAoU3RhcnQpIl07CiAgcmVzZWFyY2hlclsicmVzZWFyY2hlciJdOwogIG1hcmtldGVyWyJtYXJrZXRlciJdOwogIGxlZ2FsWyJsZWdhbCJdOwogIGFnZ3JlZ2F0b3JbImFnZ3JlZ2F0b3IiXTsKICBmYW5faW5fX2FnZ3JlZ2F0b3JfX2UzYTRmZjU4KChmYW4taW4pKQogIGxlZ2FsIC0tPiBmYW5faW5fX2FnZ3JlZ2F0b3JfX2UzYTRmZjU4OwogIG1hcmtldGVyIC0tPiBmYW5faW5fX2FnZ3JlZ2F0b3JfX2UzYTRmZjU4OwogIHJlc2VhcmNoZXIgLS0-IGZhbl9pbl9fYWdncmVnYXRvcl9fZTNhNGZmNTg7CiAgZmFuX2luX19hZ2dyZWdhdG9yX19lM2E0ZmY1OCAtLT4gYWdncmVnYXRvcjsKICBkaXNwYXRjaGVyIC0tPiByZXNlYXJjaGVyOwogIGRpc3BhdGNoZXIgLS0-IG1hcmtldGVyOwogIGRpc3BhdGNoZXIgLS0-IGxlZ2FsOw==



## Step 6: Run the Workflow

Now let's run the workflow with a sample prompt about launching a new product!

**Prompt**: "We are launching a new budget-friendly electric bike for urban commuters."

Watch as the workflow:
1. Dispatches to all three experts
2. Each expert analyzes from their perspective (in parallel)
3. Results are aggregated into a consolidated report

In [9]:
# Run the workflow
prompt = "We are launching a new budget-friendly electric bike for urban commuters."

print(f"\n{'=' * 80}")
print("RUNNING WORKFLOW")
print(f"{'=' * 80}")
print(f"\nPrompt: {prompt}\n")
print(f"{'=' * 80}\n")

# Track events
event_count = 0
agent_events = []
final_output = None

async def run_workflow():
    global event_count, agent_events, final_output
    
    async for event in workflow.run_stream(prompt):
        event_count += 1
        
        if isinstance(event, AgentRunEvent):
            agent_events.append(event)
            print(f"\n[Event #{event_count}] AgentRunEvent from: {event.executor_id}")
            # AgentRunEvent has different attributes - let's show what's available
            if hasattr(event, 'run_id'):
                print(f"  Run ID: {event.run_id}")
            if hasattr(event, 'status'):
                print(f"  Status: {event.status}")
            
        elif isinstance(event, WorkflowOutputEvent):
            final_output = event.data
            print(f"\n[Event #{event_count}] WorkflowOutputEvent - Final result received!")

# Run the async workflow
await run_workflow()

print(f"\n{'=' * 80}")
print(f"WORKFLOW COMPLETED")
print(f"{'=' * 80}")
print(f"Total events: {event_count}")
print(f"Agent events: {len(agent_events)}")


RUNNING WORKFLOW

Prompt: We are launching a new budget-friendly electric bike for urban commuters.


📤 Dispatching to 3 experts...
  → Sending to: researcher
  → Sending to: marketer
  → Sending to: legal

[Event #6] AgentRunEvent from: researcher

[Event #9] AgentRunEvent from: marketer

[Event #12] AgentRunEvent from: legal

📥 Aggregating 3 expert responses...
  ← Received from: researcher
  ← Received from: marketer
  ← Received from: legal

[Event #15] WorkflowOutputEvent - Final result received!

WORKFLOW COMPLETED
Total events: 17
Agent events: 3


## Step 7: Display Final Results

Here's the consolidated output from all three expert agents!

In [10]:
if final_output:
    print(f"\n{'=' * 80}")
    print("FINAL AGGREGATED OUTPUT")
    print(f"{'=' * 80}\n")
    print(final_output)
    print(f"\n{'=' * 80}")
else:
    print("⚠️  No final output was produced. Check the workflow execution above.")


FINAL AGGREGATED OUTPUT

Consolidated Insights

Research Findings:
**Insights:**
- Urban commuters increasingly seek cost-effective, sustainable transportation, with rising interest in electric bikes (e-bikes) due to congestion, fuel costs, and environmental concerns.
- Key purchasing factors: price, battery range, durability, portability (weight/foldability), and style.
- The global e-bike market is projected to grow at 10%+ CAGR through 2030 (Allied Market Research, 2023). Budget segment demand is high, especially in densely populated cities.

**Opportunities:**
- Capture market share among students, young professionals, and gig economy workers (e.g., delivery riders).
- Differentiate through features commonly compromised in budget models (e.g., removable batteries, reliable after-sales support, smartphone integration).
- Form partnerships with local businesses for cross-promotions or test rides; target regions with robust cycling infrastructure or supportive local policies.
- Subsc

## Summary

This notebook demonstrated:

✅ **Fan-out/Fan-in Pattern** - Dispatching work to multiple agents in parallel and aggregating results

✅ **Custom Executors** - Building specialized workflow steps (dispatcher and aggregator)

✅ **Agent Executors** - Wrapping AI agents with specific roles and instructions

✅ **Workflow Visualization** - Generating Mermaid and GraphViz diagrams

✅ **Event Streaming** - Tracking workflow execution through events

### Key Takeaways:

1. **Parallelism** - Multiple agents can work concurrently on the same input
2. **Structured Output** - Aggregator consolidates responses into organized sections
3. **Clear Visualization** - Workflow structure is easy to understand and debug
4. **Flexible Architecture** - Easy to add more experts or change the workflow structure

### Try It Yourself:

Modify the prompt in Step 6 to analyze different scenarios:
- "We're developing an AI-powered fitness app for seniors"
- "We want to expand our coffee shop chain to Asia"
- "We're launching a subscription service for sustainable fashion"

In [13]:
# Run the workflow
prompt = "We are developing an AI-powered fitness app for seniors."

print(f"\n{'=' * 80}")
print("RUNNING WORKFLOW")
print(f"{'=' * 80}")
print(f"\nPrompt: {prompt}\n")
print(f"{'=' * 80}\n")

# Track events
event_count = 0
agent_events = []
final_output = None

async def run_workflow():
    global event_count, agent_events, final_output
    
    async for event in workflow.run_stream(prompt):
        event_count += 1
        
        if isinstance(event, AgentRunEvent):
            agent_events.append(event)
            print(f"\n[Event #{event_count}] AgentRunEvent from: {event.executor_id}")
            # AgentRunEvent has different attributes - let's show what's available
            if hasattr(event, 'run_id'):
                print(f"  Run ID: {event.run_id}")
            if hasattr(event, 'status'):
                print(f"  Status: {event.status}")
            
        elif isinstance(event, WorkflowOutputEvent):
            final_output = event.data
            print(f"\n[Event #{event_count}] WorkflowOutputEvent - Final result received!")

# Run the async workflow
await run_workflow()

print(f"\n{'=' * 80}")
print(f"WORKFLOW COMPLETED")
print(f"{'=' * 80}")
print(f"Total events: {event_count}")
print(f"Agent events: {len(agent_events)}")


RUNNING WORKFLOW

Prompt: We are developing an AI-powered fitness app for seniors.


📤 Dispatching to 3 experts...
  → Sending to: researcher
  → Sending to: marketer
  → Sending to: legal

[Event #6] AgentRunEvent from: researcher

[Event #6] AgentRunEvent from: researcher

[Event #9] AgentRunEvent from: marketer

[Event #9] AgentRunEvent from: marketer

[Event #12] AgentRunEvent from: legal

📥 Aggregating 3 expert responses...
  ← Received from: researcher
  ← Received from: marketer
  ← Received from: legal

[Event #15] WorkflowOutputEvent - Final result received!

WORKFLOW COMPLETED
Total events: 17
Agent events: 3

[Event #12] AgentRunEvent from: legal

📥 Aggregating 3 expert responses...
  ← Received from: researcher
  ← Received from: marketer
  ← Received from: legal

[Event #15] WorkflowOutputEvent - Final result received!

WORKFLOW COMPLETED
Total events: 17
Agent events: 3


In [14]:
if final_output:
    print(f"\n{'=' * 80}")
    print("FINAL AGGREGATED OUTPUT")
    print(f"{'=' * 80}\n")
    print(final_output)
    print(f"\n{'=' * 80}")
else:
    print("⚠️  No final output was produced. Check the workflow execution above.")


FINAL AGGREGATED OUTPUT

Consolidated Insights

Research Findings:
**Insights:**
- Seniors represent a growing tech-savvy segment: 42% of U.S. adults 65+ use smartphones (Pew, 2023).
- Senior fitness needs are specific: low-impact exercises, fall prevention, balance, flexibility, and chronic condition management.
- AI features (personalization, voice assistants, progress tracking) can enhance engagement and adapt routines to physical limitations.

**Opportunities:**
- Differentiate with accessibility: large fonts, voice control, simple navigation, integration with wearables.
- Offer tailored programs for common conditions (arthritis, osteoporosis, diabetes), guided by AI.
- Collaborate with healthcare providers, senior living communities, and insurers for broader distribution or referrals.
- Family/caregiver or telehealth connectivity features increase value and trust.
- Gamification or social features can improve adherence and motivation.

**Risks:**
- App complexity or poor UI may d